In [ ]:
import sys
import io
import traceback
import csv
from pathlib import Path
import pandas as pd
import numpy as np 
from openpyxl import load_workbook 
import config as cfg
import eda_tools as tls

load config

In [ ]:
# configuration load
log_path = Path(Path.cwd().parent /  r"config/config.json")
if not log_path.exists(): 
    print(f"Arquivo de configuração não encontrado !\n{log_path}")
    sys.exit(1)
log = cfg.config_log(log_path)
cfg.load_config() 
pd.set_option('future.no_silent_downcasting', True)

load sheets

In [ ]:
# eda dfs create
df_tables = pd.read_excel(cfg.eda_sheet_full_path,sheet_name='tables')
df_fields = pd.read_excel(cfg.eda_sheet_full_path,sheet_name='fields', header=1)

# eda dfs config
df_fields = df_fields.astype('object') 
df_fields.columns = df_fields.columns.str.strip().str.lower()
df_tables.columns = df_tables.columns.str.strip().str.lower()

# open eda sheet workbooks
try:
    wb = load_workbook(cfg.eda_sheet_full_path, data_only=False)
    tables_workbook = wb["tables"]
    fields_workbook = wb["fields"]
except FileNotFoundError:
    print("Erro ao abrir planilhas")
    exit(1)


collect stats 

In [ ]:
# create collect structures
from eda_tools import file_stats

df_table_stats = pd.DataFrame(columns=['table','stat','value'])
table_stats_collected = []
df_field_stats = pd.DataFrame(columns=['table','field','stat','value'])
field_stats_collected = []

for index, table in df_tables.iterrows():
    table_name = table['table']
    print(f"-----{table_name}-----")

    # load data
    data_path = Path(cfg.data_file_path / table['file'])
    df_data = pd.read_csv(data_path,encoding=tls.encode(data_path),quotechar=None,quoting=3,keep_default_na=True,sep=cfg.csv_sep,engine='python')
    df_data.columns = df_data.columns.str.strip().str.lower()

    # ---------  table stats ---------  
    file_size, col_count, col_exists, col_unique = file_stats(table_name,df_fields,data_path,cfg.csv_sep)
    table_stats_collected.append({"table": table_name, "stat": "file_size", "value":file_size})
    table_stats_collected.append({"table": table_name, "stat": "lines", "value":len(df_data)})
    table_stats_collected.append({"table": table_name, "stat": "columns", "value":col_count})
    table_stats_collected.append({"table": table_name, "stat": "columns_existence", "value":col_exists})    
    table_stats_collected.append({"table": table_name, "stat": "column_unique", "value": col_unique})    

    # pk_unique	
    #table_stats_collected.append({"table": table_name, "stat": stat, "pk_unique": })    
    
    # referencial_integrity
    #table_stats_collected.append({"table": table_name, "stat": stat, "referencial_integrity": })

    # ---------  fields stats ---------

    # Counts by content type 
    df_types = df_data.map(tls.classify_content)
    df_types.apply(pd.Series.value_counts)

    for field in df_types.columns:
        counts = df_types[field].value_counts(dropna=False)
        for stat, value in counts.items():
            field_stats_collected.append({"table": table_name, "field": field, "stat": stat, "value": int(value)})

    # describe stats
    df_fields_by_table = df_fields.reset_index()
    df_fields_by_table = df_fields_by_table[df_fields_by_table['table'] == table_name]
    for idx, fld in df_fields_by_table.iterrows(): 
        field_name = fld['field']
        if fld['type'] == "str":
            field_series = df_data[field_name]             
            field_series = field_series.astype('object')            
        else: 
           field_series = pd.to_numeric(df_data[field_name], errors='coerce')
        # count 
        field_stats_collected.append({"table": table_name, "field": field_name, "stat": "count" , "value": len(df_data)})
        # min
        field_stats_collected.append({"table": table_name, "field": field_name, "stat": "min" , "value": field_series.min()})
        # max 
        field_stats_collected.append({"table": table_name, "field": field_name, "stat": 'max', "value": field_series.max()})
        # mean 
        try: 
            field_stats_collected.append({"table": table_name, "field": field_name, "stat": "mean", "value": field_series.mean()})        
        except: 
            field_stats_collected.append({"table": table_name, "field": field_name, "stat": "mean", "value": "no number"})
        # std
        try:        
            field_stats_collected.append({"table": table_name, "field": field_name, "stat":"std", "value": field_series.std()}) 
        except: 
            field_stats_collected.append({"table": table_name, "field": field_name, "stat":"std", "value": "no number"})    
        # nunique    
        field_stats_collected.append({"table": table_name, "field": field_name, "stat":"unique", "value": field_series.nunique()})
        # top
        try: 
            field_stats_collected.append({"table": table_name, "field": field_name, "stat":"top", "value": field_series.mode().iloc[0]})
        except: 
            field_stats_collected.append({"table": table_name, "field": field_name, "stat":"top", "value": "no value"})
        # freq
        try: 
            field_stats_collected.append({"table": table_name, "field": field_name, "stat":"freq", "value": field_series.value_counts().iloc[0]})
        except: 
           field_stats_collected.append({"table": table_name, "field": field_name, "stat":"freq", "value": "no value"})
        # q1, q2, q3
        try: 
            q_values = field_series.quantile([0.25, 0.5, 0.75])
            field_stats_collected.append({"table": table_name, "field": field_name, "stat":"q1(25%)", "value": q_values.iloc[0]})
            field_stats_collected.append({"table": table_name, "field": field_name, "stat":"q2(50%)", "value": q_values.iloc[1]})
            field_stats_collected.append({"table": table_name, "field": field_name, "stat":"q3(75%)", "value": q_values.iloc[2]})
        except: 
            field_stats_collected.append({"table": table_name, "field": field_name, "stat":"q1(25%)", "value": "no value"})
            field_stats_collected.append({"table": table_name, "field": field_name, "stat":"q2(50%)", "value": "no value"})
            field_stats_collected.append({"table": table_name, "field": field_name, "stat":"q3(75%)", "value": "no value"})
        # format
        try: 
            if not pd.isna(fld['regex']): 
                if fld['type'] == 'int': 
                    field_series_num = pd.to_numeric(field_series, errors='coerce')
                    field_series = field_series_num.round(0).astype('Int64')
                regex_pattern = fld['regex']
                bool_series = field_series.astype(str).str.strip().str.match(regex_pattern, na=False)
                qty_formats = bool_series.sum()
                field_stats_collected.append({"table": table_name, "field": field_name, "stat":"valid_format", "value": qty_formats})                
            else: 
                field_stats_collected.append({"table": table_name, "field": field_name, "stat":"valid_format", "value": "no format"})
        except Exception as e:
            exc_type, exc_obj, exc_tb = sys.exc_info() 
            print("🛑 Erro format!")
            print(f"📝 Tipo de Erro: {type(e).__name__}")
            linha_do_erro = exc_tb.tb_lineno            
            print(f"👉 Linha do Código que Gerou o Erro: {linha_do_erro}")
            print("\n--- Traceback Completo ---")

            field_stats_collected.append({"table": table_name, "field": field_name, "stat":"valid_format", "value": "err"})
        # list
        try: 
            if not pd.isna(fld['list']): 
                values_lst = fld['list'].split(';')
                field_series_str = field_series.astype(str)
                bool_series = field_series_str.isin(values_lst)
                field_stats_collected.append({"table": table_name, "field": field_name, "stat":"valid_on_list", "value": bool_series.sum()})                
            else: 
                field_stats_collected.append({"table": table_name, "field": field_name, "stat":"valid_on_list", "value": "no list"})                
        except Exception as e:
            exc_type, exc_obj, exc_tb = sys.exc_info() 
            print("🛑 Erro list!")
            print(f"📝 Tipo de Erro: {type(e).__name__}")
            linha_do_erro = exc_tb.tb_lineno            
            print(f"👉 Linha do Código que Gerou o Erro: {linha_do_erro}")
            print("\n--- Traceback Completo ---")

            field_stats_collected.append({"table": table_name, "field": field_name, "stat":"valid_on_list", "value": "err"})
        # range 
        try: 
            if fld['type'] == "int" or fld['type'] == "float":
                if not pd.isna(fld['range']): 
                    
                    ranges_list = fld['range']
                    range_lst = ranges_list.split(';') 
                    min_limit_str, max_limit_str = range_lst[0], range_lst[1]
                    min_limit = float(min_limit_str)
                    max_limit = float(max_limit_str)                    
                    field_series_num = pd.to_numeric(field_series, errors='coerce')
                    bool_series = field_series_num.between(min_limit, max_limit, inclusive='both')
                    qty_range = bool_series.sum()
                    field_stats_collected.append({"table": table_name, "field": field_name, "stat":"valid_on_range", "value": qty_range})                
                else: 
                    field_stats_collected.append({"table": table_name, "field": field_name, "stat":"valid_on_range", "value": "no range"})
            else: 
                field_stats_collected.append({"table": table_name, "field": field_name, "stat":"valid_on_range", "value": "no number"})
        except Exception as e: 
            exc_type, exc_obj, exc_tb = sys.exc_info()
            print("🛑 Erro range!")
            print(f"📝 Tipo de Erro: {type(e).__name__}")
            linha_do_erro = exc_tb.tb_lineno            
            print(f"👉 Linha do Código que Gerou o Erro: {linha_do_erro}")
            print("\n--- Traceback Completo ---")
            traceback.print_exc()
  
            field_stats_collected.append({"table": table_name, "field": field_name, "stat":"valid_on_range", "value": "err"})    

    # stats collected consolidation     
    df_field_stats = pd.concat([df_field_stats,pd.DataFrame(field_stats_collected)])   
    df_table_stats = pd.concat([df_table_stats,pd.DataFrame(table_stats_collected)])

df_field_stats = df_field_stats.astype('object')
df_table_stats = df_table_stats.astype('object')

In [ ]:
# update fields stats
df_field_stats_pivot = (df_field_stats.pivot_table(index=["table","field"], columns="stat", values="value", fill_value=0,aggfunc='first').reset_index())
df_field_stats_pivot.set_index(["table", "field"], inplace=True)
df_fields.set_index(["table", "field"], inplace=True)
df_fields.update(df_field_stats_pivot)

# update table stats
df_table_stats_pivot = (df_table_stats.pivot_table(index=["table"], columns="stat", values="value", fill_value=0,aggfunc='first').reset_index())
print(df_table_stats_pivot.head())
df_table_stats_pivot.set_index(["table"], inplace=True)
df_tables.set_index(["table"], inplace=True)
df_tables.update(df_table_stats_pivot)

col_list = ['nulls', 'blanks', 'int', 'float', 'str', 'date']
df_fields[col_list] = df_fields[['nulls', 'blanks', 'int', 'float', 'str', 'date']].fillna(0)


In [ ]:
# update fields workbook
df_fields = df_fields.reset_index()
col_series = df_fields.columns 

for idx, field_row in df_fields.iterrows(): 
    line = int(idx) + 3
    for col_name in col_series: 
        if col_name.startswith("unnamed"):
            continue 
        col = col_series.get_loc(col_name) + 1
        fields_workbook.cell(row=line, column=col).value = field_row[col_name]

# update table workbook
df_tables = df_tables.reset_index()
col_series = df_tables.columns 
for idx, table_row in df_tables.iterrows(): 
    line = int(idx) + 2
    for col_name in col_series: 
        if col_name.startswith("unnamed"):
            continue 
        col = col_series.get_loc(col_name) + 1
        tables_workbook.cell(row=line, column=col).value = table_row[col_name]

In [ ]:
# Save sheet
wb.save(cfg.eda_sheet_full_path)